In [1]:
# Para ID 615 from polylet
text = """
Polyrotaxane dethreading kinetics were more accurately determined by monitoring
the change in turbidity of aqueous polyrotaxane 1 and 2 dispersions at various
pHs as a function of time (Figure). Since the poorly soluble polyrotaxanes should
undergo rapid \alpha-CD dethreading in bulk aqueous solution once the endcaps
have been hydrolyzed to produce a homogeneous solution of \alpha-CD and PEG1500,
we employed this method to make a lower limit estimate of the endcap cleavage
kinetics. (Note: This is a lower limit estimate because vinyl ether hydrolysis
is a necessary first step before the subsequent unthreading process, estimated
to be on the tens-of-minutes time scale based on \alpha-CD-PEG threading rates
determined by turbidimetry, can proceed.) Turbidity analysis of polyrotaxane 1
dispersions at pH 4 produced homogeneous solutions with a half-life of 18 min
and complete degradation within 1 h (Figure A), in reasonable agreement with the
GPC kinetics analysis. Similar results were obtained for polyrotaxane 2
(Figure B). Ester hydrolysis was too rapid to be reliably detected using 1% NaOH
(solubilization occurred within a few minutes); however, a half-life of
approximately 40 min for 2 was found at pH 12.1, with complete solubilization of
2 occurring within approximately 150 min. Rate analyses yielded initial observed
pseudo-first-order rate constants of 0.0211 min^{-1} for 1 at pH 4 and 0.0122
min^{-1} for 2 at pH 12.1. Polyrotaxane 3 does not undergo appreciable
fluoride-mediated deprotection under the aqueous conditions employed in these
experiments. 6 Dethreading rates of pH-triggerable polyrotaxanes via turbidity
analysis of 1 at pH 4 (A) and 2 at pH 12.1 (B). The insets in both figures show
the first-order rate analysis of the initial phase of the dethreading reaction.
"""

In [2]:
import os
os.chdir("..")
assert 'PromptDataExtract' in os.getcwd()

In [4]:
from backend import postgres, sett
from backend.record_extraction import bert_model, record_extractor
from backend.record_extraction.utils import LoadNormalizationDataset, load_property_metadata

In [5]:
sett.load_settings()
postgres.load_settings()

Load OK: settings.yaml


In [6]:
bert = bert_model.MaterialsBERT()
bert.init_local_model(model=sett.NERPipeline.model, device=sett.NERPipeline.pytorch_device)

In [7]:
normdata = LoadNormalizationDataset(sett.DataFiles.polymer_nen_json)
norm_dataset = normdata.process_normalization_files()
prop_metadata = load_property_metadata(sett.DataFiles.properties_json)

In [8]:
ner_tags = bert.get_tags(text)
relation_extractor = record_extractor.RelationExtraction(text, ner_tags, norm_dataset, prop_metadata)
output_para, timings = relation_extractor.process_document()

In [9]:
records = output_para['material_records']
for r in records:
    print("Material:", r['material_name'])
    print("Properties:", r['property_record'])
    print("-" * 50, "\n")

Material: [{'entity_name': 'polyrotaxanes', 'material_class': 'POLYMER_FAMILY', 'role': '', 'polymer_type': '', 'normalized_material_name': '', 'coreferents': ['polyrotaxanes'], 'components': []}, {'entity_name': '- CD', 'material_class': 'POLYMER', 'role': '', 'polymer_type': 'copolymer', 'normalized_material_name': '', 'coreferents': ['- CD', 'PEG'], 'components': []}, {'entity_name': 'PEG1500', 'material_class': 'POLYMER', 'role': '', 'polymer_type': 'homopolymer', 'normalized_material_name': '', 'coreferents': ['PEG1500'], 'components': []}, {'entity_name': 'polyrotaxane', 'material_class': 'POLYMER', 'role': '', 'polymer_type': 'homopolymer', 'normalized_material_name': 'polyrotaxane', 'coreferents': ['polyrotaxane', 'Polyrotaxane', 'polyrotaxane 1'], 'components': []}]
Properties: {'entity_name': '- order rate constants', 'property_value': '0.0211 min^{-1}', 'coreferents': ['- order rate constants'], 'property_numeric_value': 0.0211, 'property_numeric_error': 0.0, 'property_value

In [11]:
records = output_para['material_records']
for r in records:
    for material in r['material_name']:
        prop = r['property_record']
        print(f"{material['entity_name']} \t--> {prop['entity_name']} = {prop['property_value']}")
    print("-" * 50)

polyrotaxanes 	--> - order rate constants = 0.0211 min^{-1}
- CD 	--> - order rate constants = 0.0211 min^{-1}
PEG1500 	--> - order rate constants = 0.0211 min^{-1}
polyrotaxane 	--> - order rate constants = 0.0211 min^{-1}
--------------------------------------------------
polyrotaxanes 	--> - order rate constants = 0.0122 
 min^{-1}
- CD 	--> - order rate constants = 0.0122 
 min^{-1}
PEG1500 	--> - order rate constants = 0.0122 
 min^{-1}
polyrotaxane 	--> - order rate constants = 0.0122 
 min^{-1}
--------------------------------------------------
